This notebook was used to create json input for the inner proofs quickly while implementing the system to test it

In [ ]:
import json
import struct
from zokrates_pycrypto.curves import Decaf377
from zokrates_pycrypto.eddsa import PrivateKey, PublicKey

def arrays_to_bytes(arrays):
    fmt = '>' + 'I' * len(arrays)  
    return struct.pack(fmt, *arrays)

def compute_net_result(consumption, production):
    p = 8444461749428370424248824938781546531375899335154063827935233455917409239041
    net_result = sum(consumption) - sum(production)
    if sum(production) > sum(consumption):
        net_result += p
    return net_result

def sign_parts(consumption, production, sk, parts=2):
    part_size = len(consumption) // parts
    signatures = []
    public_keys = []

    for i in range(parts):
        start = i * part_size
        end = (i + 1) * part_size
        raw_msg = arrays_to_bytes(consumption[start:end] + production[start:end])
        sig = sk.sign(raw_msg)
        pk = PublicKey.from_private(sk)
        signatures.append(sig)
        public_keys.append(pk)
    
    return signatures, public_keys[0]  # Assuming the same PK for all parts

if __name__ == "__main__":
    consumption = [4] * 8  # Change size as needed
    production = [2] * 8  # Change size as needed
    net_result = compute_net_result(consumption, production)

    key = 5145218980968378210524378404035776788013036277399811805365337506608398016000
    sk = PrivateKey(key, curve=Decaf377)

    parts = len(consumption) // 8
    signatures, pk = sign_parts(consumption, production, sk, parts)

   
    json_output = []
    for sig in signatures:
        json_output.extend([
            [str(sig[0].x.n), str(sig[0].y.n)], str(sig[1])
        ])

    json_output.append([str(pk.point.x.n), str(pk.point.y.n)])

    json_output.extend([
        [str(x) for x in consumption], [str(x) for x in production], str(net_result)
    ])
    print(json.dumps(json_output, indent=2))